In [1]:
import os
import sys
import numpy as np
import pandas as pd
from tensorqtl import tensorqtl, cis, post, genotypeio
import rpgQTL

In [2]:
phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed("./input/expression.bed.gz")
covariates_df = pd.read_csv("./input/covariates.txt", sep='\t', index_col=0).T
genotype_df = pd.read_csv("./input/genotype.txt.gz", sep="\t", index_col=0)
variant_df = pd.read_csv("./input/variant.txt.gz", sep="\t", index_col=0)
rpg_df = pd.read_csv("./input/region.bed", sep="\t", header=None)

In [3]:
result_dir = "./output/"
prefix = "RPG"

In [4]:
## nominal run
rpgQTL.run_nominal(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df,
    rpg_df, l_window=2000000, s_window=0, RType='remove',
    output_dir=result_dir, prefix=prefix)

cis-QTL mapping: nominal associations for all variant-phenotype pairs
  * 200 samples
  * 572 phenotypes
  * 38 covariates
  * 237916 variants


/cellar/users/jig049/miniconda3/envs/rpgQTL/lib/python3.9/site-packages/tensorqtl/core.py:53: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at /croot/pytorch-select_1717607455294/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2416.)
  self.Q_t, _ = torch.qr(C_t - C_t.mean(0))


    ** dropping 315 phenotypes without variants
  * Computing associations
    Mapping chromosome chr20
    processing phenotype 257/257
    time elapsed: 0.06 min
    * writing output
done.
    processing phenotype 257/257


In [5]:
## permutation run
egenes_df = rpgQTL.run_permutation(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df, 
    rpg_df, l_window=2000000, s_window=0, RType='remove', seed=123456)

cis-QTL mapping: empirical p-values for phenotypes
  * 200 samples
  * 572 phenotypes
  * 38 covariates
  * 237916 variants
  * using seed 123456
    ** dropping 315 phenotypes without variants
  * computing permutations
    * WARNING: excluding 6 monomorphic variants
    * WARNING: excluding 5 monomorphic variants
    * WARNING: excluding 1 monomorphic variants
    * WARNING: excluding 1 monomorphic variants
    * WARNING: excluding 3 monomorphic variants
    * WARNING: excluding 2 monomorphic variants
    * WARNING: excluding 1 monomorphic variants
    * WARNING: excluding 1 monomorphic variants
    * WARNING: excluding 3 monomorphic variants
    * WARNING: excluding 3 monomorphic variants
    * WARNING: excluding 3 monomorphic variants
    * WARNING: excluding 3 monomorphic variants
    * WARNING: excluding 3 monomorphic variants
    * WARNING: excluding 3 monomorphic variants
    * WARNING: excluding 1 monomorphic variants
    * WARNING: excluding 2 monomorphic variants
    * WARNI

In [6]:
## calculate q-values
post.calculate_qvalues(egenes_df, fdr=0.05, qvalue_lambda=0.85)
egenes_df.to_csv("%s/%s.egenes.tsv" % (result_dir, prefix), index=True)

Computing q-values
  * Number of phenotypes tested: 257
  * Correlation between Beta-approximated and empirical p-values: : 1.0000
  * Calculating q-values with lambda = 0.850
  * Proportion of significant phenotypes (1-pi0): 0.12
  * QTL phenotypes @ FDR 0.05: 38
  * min p-value threshold @ FDR 0.05: 0.00868242


/cellar/users/jig049/miniconda3/envs/rpgQTL/lib/python3.9/site-packages/tensorqtl/post.py:53: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lb = lb[-1]
/cellar/users/jig049/miniconda3/envs/rpgQTL/lib/python3.9/site-packages/tensorqtl/post.py:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ub = ub[0]


In [7]:
## significant pairs
pairs_df = post.get_significant_pairs(egenes_df, "%s/%s" % (result_dir, prefix))
pairs_df.to_csv("%s/%s.sig_pairs.tsv" % (result_dir, prefix), index=False)

[Aug 06 14:59:12] tensorQTL: filtering significant variant-phenotype pairs
  * parsing significant variant-phenotype pairs for chr. 1/1
[Aug 06 14:59:12] done


In [8]:
## independent eQTL
indep_df = rpgQTL.run_independent(genotype_df, variant_df, egenes_df, phenotype_df, phenotype_pos_df, covariates_df, 
    rpg_df, l_window=2000000, s_window=0, RType='remove', seed=123456)
indep_df.to_csv("%s/%s.indep.tsv" % (result_dir, prefix), index=False)

cis-QTL mapping: conditionally independent variants
  * 200 samples
  * 38/257 significant phenotypes
  * 38 covariates
  * 237916 variants
  * using seed 123456
  * computing independent QTLs
    processing phenotype 38/38
  Time elapsed: 0.09 min
done.
